In [4]:
%reset -f
import pandas as pd
import requests
import threading
from threading import Thread
class symbol():
    
    def binance_margin(self):
        url = 'https://api.binance.com/api/v3/exchangeInfo'

        payload={}
        files={}
        headers = {}

        pairs = requests.request("GET", url, headers=headers, data=payload, files=files)
        pairs=pairs.json()
        pairs=pairs['symbols']
        pairs=pd.DataFrame(pairs)
        i=0
        while i< len(pairs):
            if 'MARGIN' not in pairs.loc[i,'permissions']:
                pairs.drop((i), inplace=True)
                pairs.reset_index(drop=True,inplace=True)
                i-=1
            i+=1
        pairs.drop(pairs.loc[pairs['quoteAsset']!="USDT"].index, inplace=True)
        pairs.reset_index(drop=True,inplace=True)
        pairs.rename(columns={"symbol":"pair"},inplace=True)
        pairs=pairs['pair']
        pairs=pd.DataFrame(pairs)
        pairs.to_pickle("pairs_binance_margin.pkl")
        return pairs
        
    def binance_future(self):
        url = 'https://fapi.binance.com/fapi/v1/exchangeInfo'

        payload={}
        files={}
        headers = {}

        pairs = requests.request("GET", url, headers=headers, data=payload, files=files)
        pairs=pairs.json()
        pairs=pairs['symbols']
        pairs=pd.DataFrame(pairs)
        pairs.drop(pairs.loc[pairs['contractType']!='PERPETUAL'].index, inplace=True)
        pairs.drop(pairs.loc[pairs['quoteAsset']!="USDT"].index, inplace=True)
        pairs.drop(['pair'], inplace=True,axis=1)
        pairs.reset_index(drop=True,inplace=True)
        pairs.rename(columns={"symbol":"pair"},inplace=True)
        pairs=pairs['pair']
        pairs=pd.DataFrame(pairs)
        pairs.to_pickle("pairs_binance_future.pkl")
        return pairs

    def coinex_margin(self):
        url = "https://api.coinex.com/v1/margin/market"

        payload={}
        files={}
        headers = {}

        pairs = requests.request("GET", url, headers=headers, data=payload, files=files)
        pairs=pairs.json()
        pairs=pairs['data']
        pairs=pd.DataFrame([pairs.keys()]).T
        pairs = pairs.rename({0: 'pair'}, axis=1)
        i=1
        while i< len(pairs):

            t=pairs.loc[i,'pair']
            usdt=t[-4]+t[-3]+t[-2]+t[-1]
            if usdt!='USDT':
                pairs.drop(i,inplace=True)
                pairs.reset_index(drop=True,inplace=True)
                i=i-1
            i=i+1
        pairs.to_pickle("pairs_coinex_margin.pkl")
        return pairs
    
    def coinex_future(self):
        url = "https://api.coinex.com/perpetual/v1/market/list"
        payload={}
        files={}
        headers = {}

        pairs = requests.request("GET", url, headers=headers, data=payload, files=files)    
        pairs=pairs.json()  
        pairs=pairs['data']
        pairs=pd.DataFrame(pairs)
        pairs.drop(pairs.loc[pairs['money']!='USDT'].index, inplace=True)
#         pairs=pairs['stock']+'/'+pairs['money']
        pairs=pd.DataFrame(pairs)
        pairs=pairs['name']
        pairs=pd.DataFrame(pairs)
        pairs = pairs.rename({'name': 'pair'}, axis=1)
        pairs.reset_index(drop=True,inplace=True)
        pairs.to_pickle("pairs_coinex_future.pkl")
        return pairs
    
    def nobitex(self):
        url = "https://api.nobitex.ir/v2/orderbook/all"

        payload={}
        files={}
        headers = {}

        pairs = requests.request("GET", url, headers=headers, data=payload, files=files)

        pairs=pairs.json()
        pairs=list(pairs.keys())
        pairs.remove('status')
        pairs=pd.DataFrame(pairs)
        pairs.rename(columns={0:"pair"},inplace=True)
        pairs.to_pickle("pairs_nobitex.pkl")
        return pairs
    
    def kucoin_margin(self):
        url = "https://api.kucoin.com/api/v1/symbols"

        payload={}
        files={}
        headers = {}

        pairs = requests.request("GET", url, headers=headers, data=payload, files=files)
        pairs=pairs.json()
        pairs=pairs['data']
        pairs=pd.DataFrame(pairs)
        pairs.drop(pairs.loc[pairs['isMarginEnabled']!=True].index, inplace=True)
        pairs.drop(pairs.loc[pairs['quoteCurrency']!="USDT"].index, inplace=True)
        pairs.reset_index(drop=True,inplace=True)
        pairs.rename(columns={"symbol":"pair"},inplace=True)
        pairs=pairs['pair']
        pairs=pd.DataFrame(pairs)
        pairs.to_pickle("pairs_kucoin_margin.pkl")
        return pairs

    def kucoin_future(self):
        url = " https://api-futures.kucoin.com/api/v1/contracts/active"
        payload={}
        files={}
        headers = {}

        pairs = requests.request("GET", url, headers=headers, data=payload, files=files)
        pairs=pairs.json()
        pairs=pairs['data']
        pairs=pd.DataFrame(pairs)
        pairs.drop(pairs.loc[pairs['quoteCurrency']!="USDT"].index, inplace=True)
        pairs.rename(columns={"symbol":"pair"},inplace=True)
        pairs=pairs['pair']
        pairs=pd.DataFrame(pairs)
        pairs.reset_index(drop=True,inplace=True)
        pairs.to_pickle("pairs_kucoin_future.pkl")
        return pairs
    def get_symbol(self):
        if __name__ == '__main__':
            try:
                Thread(target = self.binance_margin).start()
                Thread(target = self.binance_future).start()
                Thread(target = self.coinex_margin).start()
                Thread(target = self.coinex_future).start()
                Thread(target = self.nobitex).start()
                Thread(target = self.kucoin_margin).start()
                Thread(target = self.kucoin_future).start()
            except Exception:
                    logging.error(traceback.format_exc())
                
run = symbol()
run.get_symbol()


# symbol=symbol()
# kucoin_future=symbol.nobitex()
# kucoin_future

Exception in thread Thread-15:
Traceback (most recent call last):
  File "C:\Users\hossain\AppData\Local\Programs\Python\Python37\lib\site-packages\urllib3\connection.py", line 160, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "C:\Users\hossain\AppData\Local\Programs\Python\Python37\lib\site-packages\urllib3\util\connection.py", line 84, in create_connection
    raise err
  File "C:\Users\hossain\AppData\Local\Programs\Python\Python37\lib\site-packages\urllib3\util\connection.py", line 74, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\hossain\AppData\Local\Programs\Python\Python37\lib\site-packages\urllib3\connectionpool.py", line 677, in urlopen
    chunked=chunked,
  File "C:\Users\hossain\AppData\Local\Programs\Python\Py

In [12]:
# pd.read_pickle("pairs_kucoin_future.pkl")
import datetime 
import requests
t=requests.request("GET",'https://api-futures.kucoin.com/api/v1/timestamp').json()['data']
datetime.datetime.fromtimestamp(t / 1000)
# datetime.datetime.fromtimestamp(t/1000).strftime('%H:%M:%S.%f')[:-3]
t=(datetime.datetime.utcnow() + datetime.timedelta(hours=+8)).strftime('%H:%M:%S.%f')[:-3]
tt=(datetime.datetime.utcnow() + datetime.timedelta(hours=+8)).strftime('%H:%M:%S.%f')[:-3]

'01:19:29.826'

In [11]:

import pandas as pd
import requests
# !pip install pytse-client
# !pip install pytse
url = "http://members.tsetmc.com/tsev2/excel/MarketWatchPlus.aspx?d=0"
payload={}
files={}
headers = {}

pairs = requests.request("GET", url, headers=headers, data=payload, files=files)    
pairs=pairs.csv
pairs

AttributeError: 'Response' object has no attribute 'csv'